In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import json
import folium

from sklearn import preprocessing 

In [ ]:
df = sns.load_dataset('titanic')

In [ ]:
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [ ]:
# NaN search
df.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [ ]:
# count the number of NaN in deck 
nan_deck = df['deck'].value_counts(dropna=False)
print(nan_deck)

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64


In [ ]:
df.notnull().sum(axis = 0) # axis = 1

survived       891
pclass         891
sex            891
age            714
sibsp          891
parch          891
fare           891
embarked       889
class          891
who            891
adult_male     891
deck           203
embark_town    889
alive          891
alone          891
dtype: int64

In [ ]:
# count the number of NaN with for loop
missing_df = df.isnull()
for col in missing_df.columns:
  missing_count = missing_df[col].value_counts()
  try:
     print(col, ':', missing_count[True]) # NaN값이 있으면 개수를 출력
  except:
     print(col, ':', missing_count[0])  # NaN 값이 없으면 0 출력

survived : 891
pclass : 891
sex : 891
age : 177
sibsp : 891
parch : 891
fare : 891
embarked : 2
class : 891
who : 891
adult_male : 891
deck : 688
embark_town : 2
alive : 891
alone : 891


In [ ]:
# NaN 값이 500개 이상인 열을 모두 삭제 - deck열 (891개중 688개 NaN) 삭제
df_tresh = df.dropna(axis = 1, thresh = 500)
print(len(df_tresh.columns))
print(df_tresh.columns) # deck was removed

14
Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')


In [ ]:
# age 열에 나이데이터 없는 모든행을 삭제
df_age = df.dropna(subset = ['age'], how='any', axis = 0)
df_age.isnull().sum()

KeyError: ignored

In [ ]:
# age 열의 NaN값을 다른 나의 데이터의 평균으로 변경
mean_age = df['age'].mean(axis=0) # age 열의 평균을 계산 (NAN 값을 제외)
df['age'].fillna(mean_age, inplace=True)

In [ ]:
df.info()   # age was filled.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          891 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [ ]:
# embark_town 승선지는 최빈값 (가장 많이 출연한 값)
most_freq = df['embark_town'].value_counts(dropna = True).idxmax()
print(most_freq)

Southampton


In [ ]:
df['embark_town'].fillna(most_freq, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          891 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  891 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [ ]:
# embarked 열의 NaN 값을 바로 앞에 있는 값으로 변경하기
df['embarked'].fillna(method='ffill', inplace=True)  # <-> backfill:‘bfill’, 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          891 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     891 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  891 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [ ]:
# 중복 데이터를 갖는 데이터 찾기
df2 = pd.DataFrame({'c1':['a','a','b','a','b'],
                    'c2':[1,1,1,2,2],
                    'c3':[1,1,2,2,2]
                    })
df2

,c1,c2,c3
0,a,1,1
1,a,1,1
2,b,1,2
3,a,2,2
4,b,2,2


In [ ]:
df_dup = df2.duplicated()    # By default, for each set of duplicated values, the first occurrence is set on False and all others on True.
print(df_dup)

0    False
1     True
2    False
3    False
4    False
dtype: bool


In [ ]:
# 데이터프레임의 특정열에서 데이터 중복 값을 찾기
col_dup = df2['c2'].duplicated()
print(col_dup)

0    False
1     True
2     True
3    False
4     True
Name: c2, dtype: bool


In [ ]:
# 데이터프레임에서 중복 행 제거
df3 = df2.drop_duplicates()
df3

,c1,c2,c3
0,a,1,1
2,b,1,2
3,a,2,2
4,b,2,2


In [ ]:
# c2, c3열을 기준으로 중복행 제거
df4 = df2.drop_duplicates(subset = ['c2','c3'])
df4   # 1, 4 rows are duplicates -> remove

,c1,c2,c3
0,a,1,1
2,b,1,2
3,a,2,2


# Conversion

In [ ]:
df = pd.read_csv('./auto-mpg.csv', header = None)
df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 
              'acceleration', 'model year', 'origin', 'name']
df.head()
# mpg (mile per gallon)로 kpl(kilometer per liter)을 생성 (mpg_to_kpl)
mpg_to_kpl = 1.60934 / 3.78541
print(mpg_to_kpl)
# mpg열에 0.425를 곱해서 새로운 kpl컬럼을 생성
df['kpl'] = df['mpg'] * mpg_to_kpl
df.head(3)

0.42514285110463595


,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,7.652571
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,6.377143
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,7.652571


In [ ]:
# 반올림
df['kpl'] = df['kpl'].round(2)
df.head(3)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,7.65
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,6.38
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,7.65


In [ ]:
# 각 열의 자료형 확인
df.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight          float64
acceleration    float64
model year        int64
origin            int64
name             object
kpl             float64
dtype: object

In [ ]:
# horse 열의 고유값
print(df['horsepower'].unique())

['130.0' '165.0' '150.0' '140.0' '198.0' '220.0' '215.0' '225.0' '190.0'
 '170.0' '160.0' '95.00' '97.00' '85.00' '88.00' '46.00' '87.00' '90.00'
 '113.0' '200.0' '210.0' '193.0' '?' '100.0' '105.0' '175.0' '153.0'
 '180.0' '110.0' '72.00' '86.00' '70.00' '76.00' '65.00' '69.00' '60.00'
 '80.00' '54.00' '208.0' '155.0' '112.0' '92.00' '145.0' '137.0' '158.0'
 '167.0' '94.00' '107.0' '230.0' '49.00' '75.00' '91.00' '122.0' '67.00'
 '83.00' '78.00' '52.00' '61.00' '93.00' '148.0' '129.0' '96.00' '71.00'
 '98.00' '115.0' '53.00' '81.00' '79.00' '120.0' '152.0' '102.0' '108.0'
 '68.00' '58.00' '149.0' '89.00' '63.00' '48.00' '66.00' '139.0' '103.0'
 '125.0' '133.0' '138.0' '135.0' '142.0' '77.00' '62.00' '132.0' '84.00'
 '64.00' '74.00' '116.0' '82.00']


In [ ]:
# 누락 데이터 ('?') 삭제
df['horsepower'].replace('?', np.nan, inplace=True)             
df.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model year      0
origin          0
name            0
kpl             0
dtype: int64

In [ ]:
df.dropna(subset=['horsepower'], axis = 0, inplace = True)
print(df.isnull().sum())

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
name            0
kpl             0
dtype: int64


In [ ]:
df['horsepower'] = df['horsepower'].astype(float)
df['horsepower'].dtypes

dtype('float64')

In [ ]:
# origin열에 고유값을 확인
df['origin'].unique()

array([1, 3, 2])

In [ ]:
df['origin'].replace({1:'USA', 2:'EU', 3:'JAPAN'}, inplace = True)

In [ ]:
df['origin'].unique()

array(['USA', 'JAPAN', 'EU'], dtype=object)

In [ ]:
print(df['origin'].dtypes)

object


In [ ]:
# origin 열의 문자열 자료형을 범주형으로 변환
df['origin'] = df['origin'].astype('category')
print(df['origin'].dtypes)

category


In [ ]:
df['origin'] = df['origin'].astype('str') # 다시 string으로
print(df['origin'].dtypes)

object


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight        392 non-null    float64
 5   acceleration  392 non-null    float64
 6   model year    392 non-null    int64  
 7   origin        392 non-null    object 
 8   name          392 non-null    object 
 9   kpl           392 non-null    float64
dtypes: float64(6), int64(2), object(2)
memory usage: 33.7+ KB


In [ ]:
df['model year'].sample(3) # 랜덤 샘플로 3개 뽑기

1      70
149    74
376    82
Name: model year, dtype: int64

In [ ]:
# model year 정수를 범주형으로 변환
df['model year'] = df['model year'].astype('category')
df['model year'].sample(3)

355    81
132    74
16     70
Name: model year, dtype: category
Categories (13, int64): [70, 71, 72, 73, ..., 79, 80, 81, 82]

# Horsepower를 3개 등급으로 나누기

In [ ]:
# histogram, make list for dividers of 3 bins
count, bin_dividers = np.histogram(df['horsepower'], bins=3)
print(bin_dividers)

[ 46.         107.33333333 168.66666667 230.        ]


In [ ]:
# 3개의 bins에 이름지정
bin_name = ['저출력', '보통출력', '고출력']

In [ ]:
# cut 함수로 각데이터를 3개의 bin에 할당
df['hp_bin'] = pd.cut(x=df['horsepower'], 
                      bins = bin_dividers, # 경계값 리스트
                      labels = bin_name,   # 구간별 이름
                      include_lowest=True) # 첫 경계값을 포함
print(df[['horsepower', 'hp_bin']])

     horsepower hp_bin
0         130.0   보통출력
1         165.0   보통출력
2         150.0   보통출력
3         150.0   보통출력
4         140.0   보통출력
..          ...    ...
393        86.0    저출력
394        52.0    저출력
395        84.0    저출력
396        79.0    저출력
397        82.0    저출력

[392 rows x 2 columns]


In [ ]:
# hp_bin 열의 범주형 데이터를 더미변수로 변환(원핫인코딩)
horsepower_dummies = pd.get_dummies(df['hp_bin'])
print(horsepower_dummies.head(10))

   저출력  보통출력  고출력
0    0     1    0
1    0     1    0
2    0     1    0
3    0     1    0
4    0     1    0
5    0     0    1
6    0     0    1
7    0     0    1
8    0     0    1
9    0     0    1


In [ ]:
# 원핫인코딩을 다른 패키지로 사용 가능
# 전처리를 위한 encoder 객체 만들기
label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()

In [ ]:
# label encoder로 문자열 범주를 숫자형 범주로 변환
# df = sns.load_dataset('mpg')
# df.head()

In [ ]:
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl,hp_bin
0,18.0,8,307.0,130.0,3504.0,12.0,70,USA,chevrolet chevelle malibu,7.65,보통출력
1,15.0,8,350.0,165.0,3693.0,11.5,70,USA,buick skylark 320,6.38,보통출력
2,18.0,8,318.0,150.0,3436.0,11.0,70,USA,plymouth satellite,7.65,보통출력
3,16.0,8,304.0,150.0,3433.0,12.0,70,USA,amc rebel sst,6.80,보통출력
4,17.0,8,302.0,140.0,3449.0,10.5,70,USA,ford torino,7.23,보통출력
...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790.0,15.6,82,USA,ford mustang gl,11.48,저출력
394,44.0,4,97.0,52.0,2130.0,24.6,82,EU,vw pickup,18.71,저출력
395,32.0,4,135.0,84.0,2295.0,11.6,82,USA,dodge rampage,13.60,저출력
396,28.0,4,120.0,79.0,2625.0,18.6,82,USA,ford ranger,11.90,저출력


In [ ]:
onehot_labeled = label_encoder.fit_transform(df['hp_bin'])
print(onehot_labeled)
print(type(onehot_labeled))

[1 1 1 1 1 0 0 0 0 0 0 1 1 0 2 2 2 2 2 2 2 2 2 1 2 0 0 0 0 2 2 2 2 2 2 2 2
 1 0 1 1 0 0 0 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 0 1 1 1 0 1 1 0 2 1 1 1
 1 1 2 2 2 2 2 2 2 2 0 1 1 1 1 0 1 1 1 0 0 0 2 2 2 2 2 2 1 1 0 0 2 2 2 2 2
 2 2 2 1 0 2 2 2 1 1 1 1 0 2 2 2 2 2 2 2 2 1 2 1 1 1 1 1 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 0 1 1 1 1 2 1 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2
 1 1 1 1 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 1 2 1 1 0 1 1 1 2 2 2 2 2 1 1 1
 1 1 2 2 2 0 0 0 1 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 1
 1 1 1 1 1 2 2 2 2 2 2 2 2 2 1 1 1 2 2 1 2 2 2 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 1 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2
 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 1 2 2 1 2 2 2 2 2 2 2 2]
<class 'numpy.ndarray'>


In [ ]:
# 2차원 행렬 형태
onehot_reshaped = onehot_labeled.reshape(len(onehot_labeled), 1)
print(onehot_reshaped)
print(type(onehot_reshaped))

[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [2]
 [0]
 [0]
 [0]
 [0]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [0]
 [0]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [0]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [0]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [0]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [2]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]


In [ ]:
onehot_fitted = onehot_encoder.fit_transform(onehot_reshaped)
print(onehot_fitted)
print(type(onehot_fitted))

  (0, 1)	1.0
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 1)	1.0
  (4, 1)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 2)	1.0
  (15, 2)	1.0
  (16, 2)	1.0
  (17, 2)	1.0
  (18, 2)	1.0
  (19, 2)	1.0
  (20, 2)	1.0
  (21, 2)	1.0
  (22, 2)	1.0
  (23, 1)	1.0
  (24, 2)	1.0
  :	:
  (367, 2)	1.0
  (368, 2)	1.0
  (369, 2)	1.0
  (370, 2)	1.0
  (371, 2)	1.0
  (372, 2)	1.0
  (373, 2)	1.0
  (374, 2)	1.0
  (375, 2)	1.0
  (376, 2)	1.0
  (377, 2)	1.0
  (378, 2)	1.0
  (379, 2)	1.0
  (380, 1)	1.0
  (381, 2)	1.0
  (382, 2)	1.0
  (383, 1)	1.0
  (384, 2)	1.0
  (385, 2)	1.0
  (386, 2)	1.0
  (387, 2)	1.0
  (388, 2)	1.0
  (389, 2)	1.0
  (390, 2)	1.0
  (391, 2)	1.0
<class 'scipy.sparse._csr.csr_matrix'>


# time serise

In [ ]:
df = pd.read_csv('./stock-data - stock-data.csv')

In [ ]:
df.head(4)

,Date,Close,Start,High,Low,Volume
0,2018-07-02,10100,10850,10900,10000,137977
1,2018-06-29,10700,10550,10900,9990,170253
2,2018-06-28,10400,10900,10950,10150,155769
3,2018-06-27,10900,10800,11050,10500,133548


In [ ]:
df['new_date'] = pd.to_datetime(df['Date'])
df

,Date,Close,Start,High,Low,Volume,new_date
0,2018-07-02,10100,10850,10900,10000,137977,2018-07-02
1,2018-06-29,10700,10550,10900,9990,170253,2018-06-29
2,2018-06-28,10400,10900,10950,10150,155769,2018-06-28
3,2018-06-27,10900,10800,11050,10500,133548,2018-06-27
4,2018-06-26,10800,10900,11000,10700,63039,2018-06-26
5,2018-06-25,11150,11400,11450,11000,55519,2018-06-25
6,2018-06-22,11300,11250,11450,10750,134805,2018-06-22
7,2018-06-21,11200,11350,11750,11200,133002,2018-06-21
8,2018-06-20,11550,11200,11600,10900,308596,2018-06-20
9,2018-06-19,11300,11850,11950,11300,180656,2018-06-19


In [ ]:
df.set_index('new_date', inplace=True)

In [ ]:
dates = ['2019-01-01', '2020-03-01', '2021-06-01']
ts_dates = pd.to_datetime(dates)
print(ts_dates)

DatetimeIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='datetime64[ns]', freq=None)


In [ ]:
# Timestamp를 period로 변환
pr_day = ts_dates.to_period(freq='D')  # 'D' = day
print(pr_day)
pr_month = ts_dates.to_period(freq='M')  # 'M' = month
print(pr_month)
pr_year = ts_dates.to_period(freq='A')  # 'A' = year
print(pr_year)

PeriodIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='period[D]')
PeriodIndex(['2019-01', '2020-03', '2021-06'], dtype='period[M]')
PeriodIndex(['2019', '2020', '2021'], dtype='period[A-DEC]')


In [ ]:
# timestamp의 배열 만들기 - 월간격, 월의 시작일 기준
ts_ms = pd.date_range(start = '2019-01-01', # 날짜범위 시작
                      end = None,   # 날짜범위 끝
                      periods = 6,  # 생성할 timestamp의 개수
                      freq = 'MS',   # 시간간격 MS: 월의 시작일 
                      tz = 'Asia/Seoul' # 시간대 지역
                      )
print(len(ts_ms))
print(ts_ms)

6
DatetimeIndex(['2019-01-01 00:00:00+09:00', '2019-02-01 00:00:00+09:00',
               '2019-03-01 00:00:00+09:00', '2019-04-01 00:00:00+09:00',
               '2019-05-01 00:00:00+09:00', '2019-06-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='MS')


In [ ]:
# 월간격, 월의 마지막 날 기준
ts_me = pd.date_range(start='2019-01-01',
                      periods=6,
                      freq='M',
                      tz='Asia/Seoul'
                      )
print(len(ts_me))
print(ts_me)

6
DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-02-28 00:00:00+09:00',
               '2019-03-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-05-31 00:00:00+09:00', '2019-06-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='M')


In [ ]:
# 분기(3개월), 간격, 월의 마지막날 기준
ts_3m = pd.date_range(start='2019-01-01',
                      periods=6,
                      freq='3M',
                      tz='Asia/Seoul')
print(len(ts_3m))
print(ts_3m)

6
DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-07-31 00:00:00+09:00', '2019-10-31 00:00:00+09:00',
               '2020-01-31 00:00:00+09:00', '2020-04-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='3M')
